In [19]:
from create_notebook import create_notebook_cells

In [46]:
code_cells = [
    """batch_id = "81b4316e-df1d-4761-b85b-8eb5d3cdb317"
    """,
    
    """# 
import sys
sys.path.append("../../Papermill")
from configs import config_get

import pandas as pd
import numpy as np
import time
import json
import core as co
from core.utils import setup_tracer
configs = config_get()
# configs = co.config_get()
timer = co.NotebookTimer()
start_time = time.time()
    """,
    
    """
tracer = setup_tracer(configs['app_insights']['instrumentation_key'])
span = tracer.start_span("notebook: leaseevent_placeholder_notebook")
span.set_attribute("batch_id", batch_id)
trace_context = (tracer,span)
    """,
    
    """
parameters = {'BatchId': batch_id}
source_directory = parameters["source_directory"]
source_file = parameters["source_file"]
file_path =  f'{source_directory}/{source_file}'
    """,
    """df_file_data = co.read_csv(configs,file_path,trace_context=trace_context)"""
]

In [47]:
mapped_table_verification_code = ["""additional_fields = {"BatchId": batch_id}
validation_columns = ['LeaseId','EffectiveDate']

df_mapped = co.df_map_for_sql_insert(df_file_data, stage_mapping, additional_fields, validation_columns=validation_columns,trace_context=trace_context)
df_mapped.dropna(inplace=True)
# df_mapped.drop_duplicates(inplace=True)
display(df_mapped.tail(1))
df_mapped.shape"""]

In [3]:
create_notebook_cells(code_cells,'test.ipynb')

In [5]:
from core import file_movement,services

file_server = services.FileServer()
file_move = file_movement.FileMovement(file_server)
file_move.upload_file('test','test.ipynb')

uploaded!


In [48]:
from core import generate_notebook_field
table_fields=['item1','item2','item3']
table_fields_mapping = {}
for item in table_fields:
    item_generator = generate_notebook_field.ItemDictGenerator(item)
    result = item_generator.generate_dict()
    table_fields_mapping.update(result)
table_mapping_code = [f'stage_mapping = {table_fields_mapping}']

In [49]:
combined_code_cells=code_cells + table_mapping_code +mapped_table_verification_code
    

In [50]:
create_notebook_cells(combined_code_cells,'test.ipynb')